# Train ALL 4 Classification Models (Optimized)
**Author:** G8  
**Task:** 2.1 - Train 4 CNN Models  
**Strategy:** Unfreeze top layers for ALL transfer learning models  

**Models:**
1. Custom CNN (simplified architecture)
2. ResNet50 (with unfrozen top 30 layers)
3. EfficientNet-B0 (with unfrozen top 20 layers)
4. MobileNetV2 (with unfrozen top 20 layers)

In [13]:
import os
import json
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50, EfficientNetB0, MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

np.random.seed(42)
tf.random.set_seed(42)

print("Setup complete")

print("Libraries imported!")
print(f"TensorFlow: {tf.__version__}")
print(f"GPU: {tf.config.list_physical_devices('GPU')}")

Setup complete
Libraries imported!
TensorFlow: 2.16.2
GPU: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [14]:
# Config
PROJECT_ROOT = Path.cwd().parent if 'notebooks' in str(Path.cwd()) else Path.cwd()
DATA_PATH = PROJECT_ROOT / "data" / "processed" / "single_objects"
MODELS_PATH = PROJECT_ROOT / "models" / "classification"
LOGS_PATH = PROJECT_ROOT / "logs" / "classification"

MODELS_PATH.mkdir(parents=True, exist_ok=True)
LOGS_PATH.mkdir(parents=True, exist_ok=True)

with open(PROJECT_ROOT / "data" / "class_mapping.json", 'r') as f:
    class_info = json.load(f)
NUM_CLASSES = class_info['num_classes']

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 20  # Start with 5 for testing

print(f"Classes: {NUM_CLASSES}")

Classes: 9


In [15]:
# Data generators with stronger augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,               # Increased for more face angle variety
    width_shift_range=0.25,          # Horizontal shift
    height_shift_range=0.25,         # Vertical shift
    shear_range=0.2,                 # Light shear for pose variation
    zoom_range=0.25,                 # Zoom in/out for distance changes
    horizontal_flip=True,            # Flip horizontally (faces are symmetric)
    brightness_range=[0.7, 1.35],    # Strong brightness variation for lighting
    channel_shift_range=25.0,        # Color channel shift for robustness
    fill_mode='nearest'
)

# Validation & Test: only rescale (no augmentation)
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_gen = train_datagen.flow_from_directory(
    DATA_PATH / 'train',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

val_gen = val_test_datagen.flow_from_directory(
    DATA_PATH / 'val',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

print(f"Train: {train_gen.samples}, Val: {val_gen.samples}")

Found 697 images belonging to 9 classes.
Found 148 images belonging to 9 classes.
Train: 697, Val: 148


## Model 1: Custom CNN (Simplified)

In [16]:
def build_custom_cnn():
    """Simplified custom CNN with better regularization"""
    model = models.Sequential([
        # Block 1
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Block 2
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Block 3
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Block 4
        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling2D(),
        
        # Classifier
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.0005),  # Higher LR for from-scratch
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

print("Building Custom CNN...")
model_cnn = build_custom_cnn()
print(f"Params: {model_cnn.count_params():,}")

Building Custom CNN...
Params: 1,689,481


In [ ]:
# Train Custom CNN
print("="*80)
print("TRAINING CUSTOM CNN")
print("="*80)

callbacks_cnn = [
    ModelCheckpoint(str(MODELS_PATH / 'custom_cnn_best.h5'), monitor='val_accuracy', 
                    save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
]

history_cnn = model_cnn.fit(
    train_gen, epochs=EPOCHS, validation_data=val_gen,
    callbacks=callbacks_cnn, verbose=1
)

model_cnn.save(MODELS_PATH / 'custom_cnn_last.h5')
print(f"\nBest: {max(history_cnn.history['val_accuracy']):.4f}")

TRAINING CUSTOM CNN
Epoch 1/20


2026-01-31 23:35:51.494771: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2250 - loss: 2.8928
Epoch 1: val_accuracy improved from None to 0.10135, saving model to /Users/kevin/Documents/GitHub/Python/VESKL/11.DAE/NEU/NEU_IE7615/Prj/Discriminative/G8/Project1/IE7615_Discriminative_Project/models/classification/custom_cnn_best.h5



Epoch 1: finished saving model to /Users/kevin/Documents/GitHub/Python/VESKL/11.DAE/NEU/NEU_IE7615/Prj/Discriminative/G8/Project1/IE7615_Discriminative_Project/models/classification/custom_cnn_best.h5
22/22 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - accuracy: 0.2783 - loss: 2.4611 - val_accuracy: 0.1014 - val_loss: 2.2658 - learning_rate: 5.0000e-04
Epoch 2/20
 2/22 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.4062 - loss: 2.1394

## Model 2: ResNet50 (Fine-tuned)

In [ ]:
def build_resnet50():
    """ResNet50 with top layers unfrozen for fine-tuning"""
    base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # Unfreeze top 30 layers
    base.trainable = True
    for layer in base.layers[:-30]:
        layer.trainable = False
    
    model = models.Sequential([
        base,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.0005),  # Higher for fine-tuning
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

print("Building ResNet50...")
model_resnet = build_resnet50()
trainable = sum([tf.size(w).numpy() for w in model_resnet.trainable_weights])
print(f"Trainable params: {trainable:,}")

In [ ]:
# Train ResNet50
print("="*80)
print("TRAINING RESNET50")
print("="*80)

callbacks_resnet = [
    ModelCheckpoint(str(MODELS_PATH / 'resnet50_best.h5'), monitor='val_accuracy',
                    save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
]

history_resnet = model_resnet.fit(
    train_gen, epochs=EPOCHS, validation_data=val_gen,
    callbacks=callbacks_resnet, verbose=1
)

model_resnet.save(MODELS_PATH / 'resnet50_last.h5')
print(f"\nBest: {max(history_resnet.history['val_accuracy']):.4f}")

## Model 3: EfficientNet-B0 (Fine-tuned)

In [ ]:
def build_efficientnet():
    """EfficientNet with top layers unfrozen"""
    base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # Unfreeze top 20 layers
    base.trainable = True
    for layer in base.layers[:-20]:
        layer.trainable = False
    
    model = models.Sequential([
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.0003),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

print("Building EfficientNet...")
model_eff = build_efficientnet()
trainable = sum([tf.size(w).numpy() for w in model_eff.trainable_weights])
print(f"Trainable params: {trainable:,}")

In [ ]:
# Train EfficientNet
print("="*80)
print("TRAINING EFFICIENTNET-B0")
print("="*80)

callbacks_eff = [
    ModelCheckpoint(str(MODELS_PATH / 'efficientnet_best.h5'), monitor='val_accuracy',
                    save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
]

history_eff = model_eff.fit(
    train_gen, epochs=EPOCHS, validation_data=val_gen,
    callbacks=callbacks_eff, verbose=1
)

model_eff.save(MODELS_PATH / 'efficientnet_last.h5')
print(f"\nBest: {max(history_eff.history['val_accuracy']):.4f}")

## Model 4: MobileNetV2 (Fine-tuned)

In [ ]:
def build_mobilenet():
    """MobileNetV2 with top layers unfrozen"""
    base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    
    # Unfreeze top 20 layers
    base.trainable = True
    for layer in base.layers[:-20]:
        layer.trainable = False
    
    model = models.Sequential([
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    
    model.compile(
        optimizer=Adam(learning_rate=0.0003),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

print("Building MobileNetV2...")
model_mobile = build_mobilenet()
trainable = sum([tf.size(w).numpy() for w in model_mobile.trainable_weights])
print(f"Trainable params: {trainable:,}")

In [ ]:
# Train MobileNetV2
print("="*80)
print("TRAINING MOBILENETV2")
print("="*80)

callbacks_mobile = [
    ModelCheckpoint(str(MODELS_PATH / 'mobilenet_best.h5'), monitor='val_accuracy',
                    save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
]

history_mobile = model_mobile.fit(
    train_gen, epochs=EPOCHS, validation_data=val_gen,
    callbacks=callbacks_mobile, verbose=1
)

model_mobile.save(MODELS_PATH / 'mobilenet_last.h5')
print(f"\nBest: {max(history_mobile.history['val_accuracy']):.4f}")

## Summary After small Epochs Test

In [ ]:
results = [
    {'model': 'Custom CNN', 'best_val_acc': max(history_cnn.history['val_accuracy'])},
    {'model': 'ResNet50', 'best_val_acc': max(history_resnet.history['val_accuracy'])},
    {'model': 'EfficientNet', 'best_val_acc': max(history_eff.history['val_accuracy'])},
    {'model': 'MobileNetV2', 'best_val_acc': max(history_mobile.history['val_accuracy'])},
]

df = pd.DataFrame(results).sort_values('best_val_acc', ascending=False)
print("\n" + "="*80)
print("5-EPOCH TEST RESULTS")
print("="*80)
print(df.to_string(index=False))

print("\n" + "="*80)
print("DECISION")
print("="*80)
print("\nIf all models >70% after 5 epochs:")
print("  -> Increase EPOCHS to 50 and retrain all")
print("\nIf some models <50% after 5 epochs:")
print("  -> Focus only on models with >70%")
print("  -> Adjust LR for struggling models")

best = df.iloc[0]
print(f"\nCurrent best: {best['model']} at {best['best_val_acc']:.1%}")

## After verifying small epochs work, change EPOCHS=50 and rerun